## Class03

In [1]:
import pandas as pd
import numpy as np

### Read data and drop duplicates

In [2]:
file_path = '/Users/ml/Google Drive/af/teaching/database/data/'
ibes_raw = pd.read_csv(file_path+'ibes_1976_1990_summ_both.txt',sep='\t',low_memory=False)
ibes_raw.columns = ibes_raw.columns.str.lower()
ibes_raw = ibes_raw.drop_duplicates(['ticker','statpers']).reset_index(drop=True)

In [3]:
ibes_raw.iloc[:5,:13]

,ticker,cusip,oftic,cname,statpers,measure,fiscalp,fpi,estflag,curcode,numest,numup,numdown
0,A,02742010,A,AMERN MEDIC BLDG,19831020,EPS,ANN,1,P,USD,1,0,0
1,A,02742010,A,AMERN MEDIC BLDG,19831117,EPS,ANN,1,P,USD,1,0,0
2,A,02742010,A,AMERN MEDIC BLDG,19831215,EPS,ANN,1,P,USD,1,0,0
3,A,02742010,A,AMERN MEDIC BLDG,19860116,EPS,ANN,1,P,USD,1,0,0
4,A,02742010,A,AMERN MEDIC BLDG,19860220,EPS,ANN,1,P,USD,1,0,0


### Keep US firms
This file contains both US firm and non-US firms. **usfirm** can be used to filter them out: it is US firm if **usfirm** is 1 and it is global firm if **usfirm** equals to 0.

In [4]:
ibes_raw.groupby('usfirm')[['ticker']].count()

,ticker
usfirm,
0,228834
1,453924


In [5]:
ibes_us = ibes_raw[ibes_raw['usfirm']==1].copy()
len(ibes_us)

453924

### Sample selection: keep firms with at least 60 month of numest 

In [6]:
ibes_us['n_numest'] = ibes_us.groupby('ticker')['numest'].transform('count')
ibes_us_1 = ibes_us[ibes_us['n_numest']>=60].copy()
ibes_us_1 = ibes_us_1.sort_values(['ticker','statpers']).reset_index(drop=True)
len(ibes_us_1)

344326

### Check number of unique firms

In [7]:
len(ibes_us['ticker'].unique())

6951

### Basic summary statistics

In [8]:
ibes_us_1[['numest','meanest','stdev']].describe()

,numest,meanest,stdev
count,344326.000000,3.443240e+05,2.857610e+05
mean,7.994543,1.674507e+05,5.551247e+04
std,7.634961,1.922401e+07,4.449504e+06
min,1.000000,-9.882353e+08,0.000000e+00
25%,2.000000,3.500000e-01,2.000000e-02
50%,5.000000,8.900000e-01,6.000000e-02
75%,12.000000,1.820000e+00,1.600000e-01
max,52.000000,2.861765e+09,8.176471e+08


In [9]:
ibes_us_1['year'] = (ibes_us_1['statpers']/10000).astype(int)
ibes_us_1.groupby('year')[['numest','meanest','stdev']].aggregate(['mean','median','std','min','max']).T

year                    1976          1977          1978          1979  \
numest  mean        5.692249      5.553678      5.795983      6.074493   
        median      4.000000      3.000000      3.000000      4.000000   
        std         5.465563      5.440635      5.693274      5.721052   
        min         1.000000      1.000000      1.000000      1.000000   
        max        29.000000     28.000000     28.000000     28.000000   
meanest mean       15.917728     14.478176     27.672631     30.469365   
        median      0.850000      0.980000      1.000000      1.070000   
        std       387.591476    381.469765    638.814774    706.432692   
        min        -4.330000     -3.470000   -100.000000    -12.000000   
        max     15282.350000  19411.750000  22323.520000  23523.520000   
stdev   mean        0.747380      0.426487      0.689023      1.522587   
        median      0.050000      0.040000      0.040000      0.050000   
        std        18.388510      9.163009     17.621033     35.940588   
        min         0.000000      0.000000      0.000000      0.000000   
        max       735.000000    345.000000    882.350000   1695.000000   

year                    1980          1981          1982          1983  \
numest  mean        6.324035      7.092612      7.719291      7.655167   
        median      4.000000      5.000000      5.000000      5.000000   
        std         5.568066      6.070099      6.889808      7.023749   
        min         1.000000      1.000000      1.000000      1.000000   
        max        28.000000     31.000000     34.000000     36.000000   
meanest mean       32.559812     31.528427     36.560658     35.041802   
        median      1.060000      1.090000      0.950000      0.840000   
        std       779.285898    781.877931    845.769494    807.799582   
        min       -87.910000  -2330.000000   -500.000000  -5495.000000   
        max     25665.970000  27935.280000  32929.390000  29399.980000   
stdev   mean        3.742299      4.077995      4.235661      4.607995   
        median      0.060000      0.070000      0.080000      0.070000   
        std        88.022080     91.869436     95.290328     98.661827   
        min         0.000000      0.000000      0.000000      0.000000   
        max      4239.100000   4706.810000   7474.530000   5664.710000   

year                    1984          1985          1986          1987  \
numest  mean        7.715934  8.418741e+00  8.942773e+00  9.299843e+00   
        median      5.000000  5.000000e+00  6.000000e+00  6.000000e+00   
        std         7.385330  8.110911e+00  8.399494e+00  8.514879e+00   
        min         1.000000  1.000000e+00  1.000000e+00  1.000000e+00   
        max        39.000000  4.400000e+01  5.200000e+01  4.400000e+01   
meanest mean       31.492565  9.251836e+04  9.528349e+05  8.535299e+05   
        median      0.900000  8.200000e-01  7.400000e-01  7.700000e-01   
        std      1014.431228  1.590610e+07  4.768792e+07  4.105745e+07   
        min    -18256.000000 -2.277600e+04 -4.308000e+03 -5.400000e+02   
        max     64411.760000  2.735294e+09  2.861765e+09  2.479412e+09   
stdev   mean        4.259071  6.948415e+00  1.017789e+05  1.357483e+05   
        median      0.060000  7.000000e-02  7.000000e-02  7.000000e-02   
        std       124.492229  2.202207e+02  5.339098e+06  6.286256e+06   
        min         0.000000  0.000000e+00  0.000000e+00  0.000000e+00   
        max      7008.000000  1.492400e+04  4.117647e+08  3.382350e+08   

year                    1988          1989          1990  
numest  mean    9.701810e+00  1.028249e+01  1.033535e+01  
        median  7.000000e+00  7.000000e+00  7.000000e+00  
        std     8.877915e+00  9.261380e+00  9.135582e+00  
        min     1.000000e+00  1.000000e+00  1.000000e+00  
        max     4.800000e+01  5.100000e+01  5.000000e+01  
meanest mean    2.114121e+05 -6.042236e+04  2.177269e+04  
        median  8.600000e-01  8.

### Percentile

In [10]:
for i in range(10,91,10):
    ibes_us_1['p'+str(i)] = ibes_us_1.groupby('year')['meanest'].transform(lambda x: x.quantile(i/100))

### Correlation

In [11]:
ibes_us_1[['numest','meanest','stdev']].corr()

,numest,meanest,stdev
numest,1.000000,-0.002907,-0.006359
meanest,-0.002907,1.000000,0.515212
stdev,-0.006359,0.515212,1.000000
